#Initial Setup

In [2]:
pip install transformers

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.0/7.0 MB 96.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 200.1/200.1 kB 29.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 103.8 MB/s eta 0:00:00


In [3]:
#import libraries
import numpy as np
import pandas as pd
from tqdm import tqdm

import matplotlib.pyplot as plt
import seaborn as sns
import plotly.express as px
from plotly.subplots import make_subplots
import plotly.graph_objects as go
%matplotlib inline

In [4]:
!cat /proc/meminfo

MemTotal:       87538728 kB
MemFree:        82909388 kB
MemAvailable:   85889664 kB
Buffers:          364192 kB
Cached:          3281632 kB
SwapCached:            0 kB
Active:           660620 kB
Inactive:        3535628 kB
Active(anon):       1128 kB
Inactive(anon):   530120 kB
Active(file):     659492 kB
Inactive(file):  3005508 kB
Unevictable:           0 kB
Mlocked:               0 kB
SwapTotal:             0 kB
SwapFree:              0 kB
Dirty:            135880 kB
Writeback:             0 kB
AnonPages:        550244 kB
Mapped:           270548 kB
Shmem:              1672 kB
KReclaimable:     125612 kB
Slab:             184016 kB
SReclaimable:     125612 kB
SUnreclaim:        58404 kB
KernelStack:        7248 kB
PageTables:         7480 kB
NFS_Unstable:          0 kB
Bounce:                0 kB
WritebackTmp:          0 kB
CommitLimit:    43769364 kB
Committed_AS:    4159488 kB
VmallocTotal:   34359738367 kB
VmallocUsed:       92024 kB
VmallocChunk:          0 kB
Percpu:          

In [ ]:
!nvidia-smi

#Data load and prep

In [5]:
from google.colab import drive
drive.mount('/content/drive')
path = '/content/drive/MyDrive/266Project/'

Mounted at /content/drive


In [6]:
raw_data = pd.read_csv(path+'final_quotes.csv')
raw_data = raw_data.dropna().drop_duplicates()
print(raw_data.shape)
raw_data.head(10)

(1913815, 3)


,quote,auth,tags
0,"I'm selfish, impatient and a little insecure. ...",Marilyn Monroe,best
1,"I'm selfish, impatient and a little insecure. ...",Marilyn Monroe,life
2,"I'm selfish, impatient and a little insecure. ...",Marilyn Monroe,love
3,"I'm selfish, impatient and a little insecure. ...",Marilyn Monroe,misattributed marilyn monroe
4,"I'm selfish, impatient and a little insecure. ...",Marilyn Monroe,mistakes
5,"I'm selfish, impatient and a little insecure. ...",Marilyn Monroe,out of control
6,"I'm selfish, impatient and a little insecure. ...",Marilyn Monroe,truth
7,"I'm selfish, impatient and a little insecure. ...",Marilyn Monroe,worst
8,You've gotta dance like there's nobody watchin...,William W. Purkey,heaven
9,You've gotta dance like there's nobody watchin...,William W. Purkey,hurt


In [7]:
#combine author and tag to create quote_context, drop quotes longer than 50 words
#data['quote_context'] = "Person " + data['auth'] + ' Context ' +data['tags']
raw_data['quote_context'] = "Write a quote about " + raw_data['tags'] + " from the perspective of " +raw_data['auth'] 
raw_data = raw_data[raw_data['quote'].str.split().apply(len) <= 50]
raw_data.head(10)

,quote,auth,tags,quote_context
0,"I'm selfish, impatient and a little insecure. ...",Marilyn Monroe,best,Write a quote about best from the perspective ...
1,"I'm selfish, impatient and a little insecure. ...",Marilyn Monroe,life,Write a quote about life from the perspective ...
2,"I'm selfish, impatient and a little insecure. ...",Marilyn Monroe,love,Write a quote about love from the perspective ...
3,"I'm selfish, impatient and a little insecure. ...",Marilyn Monroe,misattributed marilyn monroe,Write a quote about misattributed marilyn monr...
4,"I'm selfish, impatient and a little insecure. ...",Marilyn Monroe,mistakes,Write a quote about mistakes from the perspect...
5,"I'm selfish, impatient and a little insecure. ...",Marilyn Monroe,out of control,Write a quote about out of control from the pe...
6,"I'm selfish, impatient and a little insecure. ...",Marilyn Monroe,truth,Write a quote about truth from the perspective...
7,"I'm selfish, impatient and a little insecure. ...",Marilyn Monroe,worst,Write a quote about worst from the perspective...
8,You've gotta dance like there's nobody watchin...,William W. Purkey,heaven,Write a quote about heaven from the perspectiv...
9,You've gotta dance like there's nobody watchin...,William W. Purkey,hurt,Write a quote about hurt from the perspective ...


In [ ]:
raw_data.head(50)

In [8]:
data = raw_data[['quote_context','quote']]
data = data.sample(frac=0.1)
#data = data.sample(frac=0.0025)
print(data.shape)
data.head()

(165435, 2)


,quote_context,quote
971847,Write a quote about duty from the perspective ...,The dead cannot cry out for justice. It is a d...
2407125,Write a quote about ai from the perspective of...,Smart CEOs should be thinking about AI and its...
1336254,Write a quote about inefficient from the persp...,I've seen the government up close and personal...
1197192,Write a quote about gourmet from the perspecti...,I always thought it was sad that you couldn't ...
1415346,Write a quote about labeled from the perspecti...,There are many great wine producers from all o...


In [9]:
from sklearn.model_selection import train_test_split
#split data into train/validation and test for evaluation 
data, test = train_test_split(data,test_size=0.10, shuffle=True)
print(data.shape)
print(test.shape)

(148891, 2)
(16544, 2)


In [106]:
#write prepared dataset to drive for reference
data.to_csv(r'/content/drive/MyDrive/266Project/quotes.csv', header=None, index=None, sep=',', mode='w')
test.to_csv(r'/content/drive/MyDrive/266Project/test_data.csv', header=None, index=None, sep=',', mode='w')

#Preprocessing

In [52]:
from transformers import GPT2Tokenizer, GPT2LMHeadModel

MODEL_NAME = 'distilgpt2' 

tokenizer = GPT2Tokenizer.from_pretrained(MODEL_NAME)
model = GPT2LMHeadModel.from_pretrained(MODEL_NAME)

In [53]:
# Declare special tokens for padding and separating the context from the quote:
SPECIAL_TOKENS_DICT = {
    'pad_token': '<pad>',
    'additional_special_tokens': ['<quote_context>', '<quote>'],
}

# Add these special tokens to the vocabulary and resize model's embeddings:
tokenizer.add_special_tokens(SPECIAL_TOKENS_DICT)
model.resize_token_embeddings(len(tokenizer))

# Show the full list of special tokens:
print(tokenizer.special_tokens_map)

{'bos_token': '<|endoftext|>', 'eos_token': '<|endoftext|>', 'unk_token': '<|endoftext|>', 'pad_token': '<pad>', 'additional_special_tokens': ['<quote_context>', '<quote>']}


In [12]:
import csv

import torch
from torch.utils.data import Dataset


class QuoteDataset(Dataset):
  def __init__(self, filename, tokenizer, seq_length=150):

    quote_context_tkn = tokenizer.additional_special_tokens_ids[0]
    quote_tkn = tokenizer.additional_special_tokens_ids[1]
    pad_tkn = tokenizer.pad_token_id
    eos_tkn = tokenizer.eos_token_id

    self.examples = []
    with open(filename) as csvfile:
      reader = csv.reader(csvfile)
      for row in reader:
      
        # Build the quote_context and quote segments:
        quote_context = [quote_context_tkn] + tokenizer.encode(row[0], max_length=seq_length//2-1)
        quote = [quote_tkn] + tokenizer.encode(row[1], max_length=seq_length//2-2) + [eos_tkn]
        
        # Concatenate the two parts together:
        tokens = quote_context + quote + [pad_tkn] * ( seq_length - len(quote_context) - len(quote) )

        # Annotate each token with its corresponding segment:
        segments = [quote_context_tkn] * len(quote_context) + [quote_tkn] * ( seq_length - len(quote_context) )

        # Ignore the quote_context, padding, and <quote> tokens by setting their labels to -100
        labels = [-100] * (len(quote_context)+1) + quote[1:] + [-100] * ( seq_length - len(quote_context) - len(quote) )

        # Add the preprocessed example to the dataset
        self.examples.append((tokens, segments, labels))

  def __len__(self):
    return len(self.examples)

  def __getitem__(self, item):
    return torch.tensor(self.examples[item])


# Build the dataset and display the dimensions of the 1st batch for verification:
quote_dataset = QuoteDataset('quotes.csv', tokenizer)
print(next(iter(quote_dataset)).size())

Truncation was not explicitly activated but `max_length` is provided a specific value, please use `truncation=True` to explicitly truncate examples to max length. Defaulting to 'longest_first' truncation strategy. If you encode pairs of sequences (GLUE-style) with the tokenizer you can select this strategy more precisely by providing a specific strategy to `truncation`.


torch.Size([3, 150])


In [13]:
len(quote_dataset)

148891

In [14]:
import math, random

from torch.utils.data import DataLoader, Dataset
from torch.utils.data.sampler import SubsetRandomSampler


# Create data indices for training and validation splits:

indices = list(range(len(quote_dataset)))

#random.seed(22)
random.shuffle(indices)

split = math.floor(0.1 * len(quote_dataset))
train_indices, val_indices = indices[split:], indices[:split]

# Build the PyTorch data loaders:

train_sampler = SubsetRandomSampler(train_indices)
val_sampler = SubsetRandomSampler(val_indices)

train_loader = DataLoader(quote_dataset, batch_size=32, sampler=train_sampler)
val_loader = DataLoader(quote_dataset, batch_size=32, sampler=val_sampler)



#The Model

##Build Model

In [16]:
import numpy as np
from tqdm import tqdm
import torch.nn


def fit(model, optimizer, train_dl, val_dl, epochs=1, device=torch.device('cuda')):

  for i in range(epochs):

    print('\n--- Starting epoch #{} ---'.format(i))

    model.train()

    # These 2 lists will keep track of the batch losses and batch sizes over one epoch:
    losses = []
    nums = []

    for xb in tqdm(train_dl, desc="Training"):
      # Move the batch to the training device:
      inputs = xb.to(device)

      # Call the model with the token ids, segment ids, and the ground truth (labels)
      outputs = model(inputs[:,0,:], token_type_ids=inputs[:,1,:], labels=inputs[:,2,:])
      
      # Add the loss and batch size to the list:
      loss = outputs[0]

      losses.append(loss.item())
      nums.append(len(xb))

      model.zero_grad()
      loss.backward()
      optimizer.step()

    # Compute the average cost over one epoch:
    train_cost = np.sum(np.multiply(losses, nums)) / sum(nums)
    train_ppl = torch.exp(torch.tensor(train_cost))
    
#    print(outputs[0], outputs[1],outputs[2])
    # loss_c = torch.nn.CrossEntropyLoss(outputs[0],inputs[:,2,:])
    # print("loss_c", loss_c)

    # Now run the validation:

    model.eval()
    
    with torch.no_grad():
      losses = []
      nums = []

      for xb in tqdm(val_dl, desc="Validation"):
        inputs = xb.to(device)
        outputs = model(inputs[:,0,:], token_type_ids=inputs[:,1,:], labels=inputs[:,2,:])
        losses.append(outputs[0].item())
        nums.append(len(xb))

    val_cost = np.sum(np.multiply(losses, nums)) / sum(nums)
    val_ppl = torch.exp(torch.tensor(val_cost))

    print('\n--- Epoch #{} finished --- \n Training loss: {} \n Training Perplexity: {} \n Validation loss: {} \n Validation Perplexity: {}'.format(i, train_cost, train_ppl, val_cost, val_ppl))


##Train Model

In [17]:
from transformers import AdamW

# Move the model to the GPU:
device = torch.device('cuda:0')

model.to(device)

# Fine-tune GPT2 for two epochs:
optimizer = AdamW(model.parameters())
fit(model, optimizer, train_loader, val_loader, epochs=3, device=device)

/usr/local/lib/python3.9/dist-packages/transformers/optimization.py:391: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(



--- Starting epoch #0 ---


Validation: 100%|██████████| 466/466 [00:28<00:00, 16.26it/s]



--- Epoch #0 finished --- 
 Training loss: 3.8395223954323825 
 Training Perplexity: 46.50325896621431 
 Validation loss: 3.6526239027673704 
 Validation Perplexity: 38.57575239412499

--- Starting epoch #1 ---


Validation: 100%|██████████| 466/466 [00:28<00:00, 16.25it/s]



--- Epoch #1 finished --- 
 Training loss: 3.521841448183652 
 Training Perplexity: 33.8466980537957 
 Validation loss: 3.6352521201363097 
 Validation Perplexity: 37.91140992216956

--- Starting epoch #2 ---


Validation: 100%|██████████| 466/466 [00:28<00:00, 16.29it/s]


--- Epoch #2 finished --- 
 Training loss: 3.2692177640176725 
 Training Perplexity: 26.290765714653727 
 Validation loss: 3.6078238546492005 
 Validation Perplexity: 36.88569677577374


In [18]:
import torch.nn
path1 = "/content/drive/MyDrive/266Project/gpt2.pt" #right one

# Save
torch.save(model, path1)

In [67]:
#Load
model = torch.load(path1)
model.eval()

GPT2LMHeadModel(
  (transformer): GPT2Model(
    (wte): Embedding(50260, 768)
    (wpe): Embedding(1024, 768)
    (drop): Dropout(p=0.1, inplace=False)
    (h): ModuleList(
      (0-5): 6 x GPT2Block(
        (ln_1): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
        (attn): GPT2Attention(
          (c_attn): Conv1D()
          (c_proj): Conv1D()
          (attn_dropout): Dropout(p=0.1, inplace=False)
          (resid_dropout): Dropout(p=0.1, inplace=False)
        )
        (ln_2): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
        (mlp): GPT2MLP(
          (c_fc): Conv1D()
          (c_proj): Conv1D()
          (act): NewGELUActivation()
          (dropout): Dropout(p=0.1, inplace=False)
        )
      )
    )
    (ln_f): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
  )
  (lm_head): Linear(in_features=768, out_features=50260, bias=False)
)

# Model Evaluation

##Test Perplexity

In [71]:
#calculate perplexity on test data
test_dataset = QuoteDataset('test_data.csv', tokenizer)
test_loader = DataLoader(test_dataset, batch_size=32)

loss_c = torch.nn.CrossEntropyLoss()

model.eval()

with torch.no_grad():
  losses = []
  nums = []

  for xb in tqdm(test_loader, desc="Test"):
    inputs = xb.to(device)
    outputs = model(inputs[:,0,:], token_type_ids=inputs[:,1,:], labels=inputs[:,2,:])
    losses.append(outputs[0].item())
    nums.append(len(xb))

test_cost = np.sum(np.multiply(losses, nums)) / sum(nums)
test_ppl = torch.exp(torch.tensor(test_cost))

print('\nTest loss: {} \n Test Perplexity: {}'.format(test_cost, test_ppl))


Truncation was not explicitly activated but `max_length` is provided a specific value, please use `truncation=True` to explicitly truncate examples to max length. Defaulting to 'longest_first' truncation strategy. If you encode pairs of sequences (GLUE-style) with the tokenizer you can select this strategy more precisely by providing a specific strategy to `truncation`.
Test: 100%|██████████| 517/517 [00:32<00:00, 16.14it/s]


Test loss: 3.601858137193455 
 Test Perplexity: 36.666302204346806


##Bleu & Gleu Scores

In [78]:
test.head()

,quote_context,quote
677557,Write a quote about hope from the perspective ...,"If we have the capacity to make mistakes, then..."
2173234,Write a quote about test from the perspective ...,Every man has a right to utter what he thinks ...
1492692,Write a quote about malls from the perspective...,"I love consumerism, TV culture, shopping malls..."
1320974,Write a quote about impression from the perspe...,I want people to have a good impression of Isr...
1254670,Write a quote about heavyweight champion from ...,"If I can be heavyweight champion of boxing, I ..."


In [139]:
test_sample = test.sample(n=400)
test_sample.shape

(400, 2)

In [152]:
#build the code to evaluate test dataset on model and append decoded results
from nltk.translate.bleu_score import sentence_bleu
from nltk.translate.gleu_score import sentence_gleu
from nltk.translate.bleu_score import SmoothingFunction

from tqdm import tqdm
import time

# temperature_values = [0.5,1.0,2.0]
# no_repeat_ngram_size_values = [2,3]
# top_p_values = [0.00,0.75]
# top_k_values = [0,10]

# #Set1
# temperature_values=0.5
# no_repeat_ngram_size_values=2
# top_p_values=0.00
# top_k_values=0

# Set2
# temperature_values=0.5
# no_repeat_ngram_size_values=3
# top_p_values=0.75
# top_k_values=10

#Set3
# temperature_values=1
# no_repeat_ngram_size_values=3
# top_p_values=0.75
# top_k_values=10

#Set4
# temperature_values=1
# no_repeat_ngram_size_values=2
# top_p_values=0
# top_k_values=0

#Set5
# temperature_values=2
# no_repeat_ngram_size_values=2
# top_p_values=0
# top_k_values=0

#Set6
temperature_values=2
no_repeat_ngram_size_values=3
top_p_values=0.75
top_k_values=10

for i in tqdm(range(10), disable=True):
    time.sleep(1)

gpt2_quotes = []
bleu=[]
gleu=[]

for i in test_sample.index: 
    prompt = (test_sample.loc[i, "quote_context"])
    label = (test_sample.loc[i, "quote"])
    output_quote = quote_generate(prompt,temperature=a,no_repeat_ngram_size = b,top_p = c, top_k = d)
    gpt2_quotes.append(output_quote)

    label = label.split()
    output_quote = output_quote.split()

    bleu.append(sentence_bleu(label, output_quote,smoothing_function=SmoothingFunction().method1))
    gleu.append(sentence_gleu(label, output_quote))


test_sample['generated_quotes_6'] = gpt2_quotes
test_sample['bleu_score_6'] = bleu
test_sample['gleu_score_6'] = gleu

In [ ]:
test_sample

In [143]:
print("blue: ",test_sample['bleu_score_1'].mean())
print("glue: ",test_sample['gleu_score_1'].mean())

blue:  0.004931791503120119
glue:  0.0070939157730375135


In [145]:
print("blue: ",test_sample['bleu_score_2'].mean())
print("glue: ",test_sample['gleu_score_2'].mean())

blue:  0.005308427125209862
glue:  0.0075659000734117


In [147]:
print("blue: ",test_sample['bleu_score_3'].mean())
print("glue: ",test_sample['gleu_score_3'].mean())

blue:  0.005163721147192419
glue:  0.007534519838958542


In [149]:
print("blue: ",test_sample['bleu_score_4'].mean())
print("glue: ",test_sample['gleu_score_4'].mean())

blue:  0.004906697492896218
glue:  0.006864878062376614


In [151]:
print("blue: ",test_sample['bleu_score_5'].mean())
print("glue: ",test_sample['gleu_score_5'].mean())

blue:  0.005079563257195825
glue:  0.007173620533110315


In [153]:
print("blue: ",test_sample['bleu_score_6'].mean())
print("glue: ",test_sample['gleu_score_6'].mean())

blue:  0.004892647699674179
glue:  0.006969261834234911


In [154]:
test_sample.to_csv(r'/content/drive/MyDrive/266Project/test_scores.csv', sep=',', mode='w')

#Generate quote

In [155]:
#customized decoding for distilgpt2 
def quote_generate(quote_context,temperature,no_repeat_ngram_size,top_p,top_k,num_beams=5):

    context_tkn = tokenizer.additional_special_tokens_ids[0]
    quote_tkn = tokenizer.additional_special_tokens_ids[1]

    input_ids = [context_tkn] + tokenizer.encode(prompt)

    segments = [quote_tkn] * 150
    segments[:len(input_ids)] = [context_tkn] * len(input_ids)

    input_ids += [quote_tkn]

    context = torch.tensor(input_ids, dtype=torch.long, device=device)
    context = context.unsqueeze(0)

    # Move the model back to the CPU for inference:
    #model.to(torch.device('cpu'))
    model.to(torch.device('cuda'))

    test_output = model.generate(context,
                                do_sample=True,   
                                min_length=0, 
                                max_length=50,
                                top_k=top_k,                                 
                                top_p=top_p,         
                                temperature=temperature, 
                                num_beams = num_beams, 
                                no_repeat_ngram_size=no_repeat_ngram_size,
                                early_stopping=True,
                                repetition_penalty=2.0,
                                pad_token_id=tokenizer.eos_token_id)


    #decode the output
    quote = tokenizer.decode(test_output.squeeze().tolist())
    quote = quote.split('<|endoftext|>')[0].split('<quote>')[1]

    return quote

In [179]:
prompt = "Write a quote about being headstrong from the perspective of Michelle Williams "
#Write a quote about love from the perspective of Marilyn Monroe"

#best bleu and glue score 
temperature_values=0.5
no_repeat_ngram_size_values=3
top_p_values=0.75
top_k_values=10

output = quote_generate(prompt,temperature=temperature_values,
                        no_repeat_ngram_size = no_repeat_ngram_size_values,top_p = top_p_values, top_k = top_k_values)
print(output)


 I'm not saying I'm going to be an actress, but I'm saying that I'm not.


In [ ]:
I'm not just an actress, but I'm also a part-time storyteller.

#Baseline Pretrained LLM

In [ ]:
MODEL_NAME = 'distilgpt2' 
model = GPT2LMHeadModel.from_pretrained(MODEL_NAME)

In [ ]:
context_tkn = tokenizer.additional_special_tokens_ids[0]
quote_tkn = tokenizer.additional_special_tokens_ids[1]

input_ids = [context_tkn] + tokenizer.encode(prompt)

segments = [quote_tkn] * 150
segments[:len(input_ids)] = [context_tkn] * len(input_ids)

input_ids += [quote_tkn]

context = torch.tensor(input_ids, dtype=torch.long, device=device)
context = context.unsqueeze(0)

# Move the model back to the CPU for inference:
#model.to(torch.device('cpu'))
model.to(torch.device('cuda'))

test_output = model.generate(context)
